**Instalar Librerias**


In [ ]:
#%%capture
#title Setup Environment
# Install the latest Tensorflow version.
!pip3 install tensorflow_text
#!pip3 install --upgrade tensorflow-gpu
!pip3 install tensorflow-hub
!pip3 install simpleneighbors
!pip3 install nltk
!pip install gcsfs
!pip3 install geopandas
!pip3 install googlemaps
!pip3 install geocoder
!pip3 install Shapely

     |████████████████████████████████| 3.4MB 6.1MB/s 
     |████████████████████████████████| 184kB 7.3MB/s 
     |████████████████████████████████| 1.3MB 9.7MB/s 
     |████████████████████████████████| 112kB 23.9MB/s 
     |████████████████████████████████| 296kB 20.0MB/s 
     |████████████████████████████████| 143kB 24.6MB/s 
     |████████████████████████████████| 1.0MB 5.7MB/s 
     |████████████████████████████████| 6.5MB 12.8MB/s 
     |████████████████████████████████| 15.3MB 254kB/s 
  Created wheel for googlemaps: filename=googlemaps-4.4.5-cp37-none-any.whl size=37983 sha256=178891f07fc7b2d3bb61d4abe2f42631aaa672ee5d4aa5850dddb5be14585970
  Stored in directory: /root/.cache/pip/wheels/fb/24/33/51b0faba1170f6519eae3c51b6adbd9939296c73f54230b069
Successfully built googlemaps
     |████████████████████████████████| 102kB 3.9MB/s 


**Importar Librerias**

In [ ]:
import json
import nltk
import os
import pprint
import random
import simpleneighbors
import urllib
from IPython.display import HTML, display
from math import sqrt
#import tensorflow.compat.v2 as tf
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer

#nltk.download('punkt')
import pandas as pd
import numpy as np
import re
from sklearn import metrics



import geopandas
import googlemaps
import geocoder
import json
import math
import requests
from shapely.geometry import Point, Polygon

In [ ]:
print(tf.__version__)

2.4.1


**Aux Fun**

In [ ]:
def output_with_highlight(text, highlight):
  output = "<li> "
  i = text.find(highlight)
  while True:
    if i == -1:
      output += text
      break
    output += text[0:i]
    output += '<b>'+text[i:i+len(highlight)]+'</b>'
    text = text[i+len(highlight):]
    i = text.find(highlight)
  return output + "</li>\n"

def display_nearest_neighbors(query_text, num_results=5, answer_text=None):
  query_embedding = model.signatures['question_encoder'](tf.constant([query_text]))['outputs'][0]
  search_results = index.nearest(query_embedding, n=num_results)

  if answer_text:
    result_md = '''
    <p>Random Question from SQuAD:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    <p>Answer:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    ''' % (query_text , answer_text)
  else:
    result_md = '''
    <p>Job description:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    ''' % query_text

  result_md += '''
    <p>Retrieved profiles :
    <ol>
  '''

  if answer_text:
    for s in search_results:
      result_md += output_with_highlight(s, answer_text)
  else:
    for s in search_results:
      profile_desc_str = motivo_id_to_motivo_name[s]
      #result_md += '<li>' + s + '</li>\n'
      result_md += '<li>' + profile_desc_str + '</li>\n'

  result_md += "</ol>"
  display(HTML(result_md))


#-------------------------------------------------------------------------------
# Get nearest motives
def get_nearest_motives(query_text, num_results=5):
  query_embedding = model.signatures['question_encoder'](tf.constant([query_text]))['outputs']
  print("np.shape(query_embedding): {}".format(np.shape(query_embedding)))
  #search_results = index.nearest(query_embedding, n=num_results)
  return -1

#-------------------------------------------------------------------------------
def extract_main_and_context(description):  
  # Get the description length in words
  description_words = description.split()
  #n_words = len(description_words)
  #partitions = description.partition("\n")

  description = description.strip()
  s1 = re.search(r'(.*)(\n+)(.*)', description, flags=re.MULTILINE)
  
  # if have two or more paragraphs
  if s1:
    main = s1.group(1)
    context = s1.group(3)
  else:
    main = description
    context = ''

  return main, context 

**Armar La base de entrenamientos**

In [ ]:
data = pd.read_csv("Ejemplo_base.csv", engine= 'python')
#data = pd.read_csv("Ejemplo_3.csv", engine= 'python')

In [ ]:
data.head()

,motivo_name,motivo_id,Pais,Genero
0,Soda Stereo,1,Argentina,Rock
1,Gustavo Cerati,2,Argentina,Rock
2,Lenny Kravitz,3,Estados Unidos,Rock
3,The Doors,4,Estados Unidos,Rock
4,Kraken,5,Colombia,Rock


**Get embedder model**

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3" #param ["https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3", "https://tfhub.dev/google/universal-sentence-encoder-qa/3"]
model = hub.load(module_url)

**Buil index of candidates descriptions**

In [ ]:
#title Compute embeddings and build simpleneighbors index
#%%time
#from google.colab import output
import time

# Person ID to profile description
motivo_id_to_motivo_name = {}

# Build index object
#main, context = extract_main_and_context(profile_descriptions[0])
encodings = model.signatures['response_encoder'](
  input=tf.constant(["Dummy text"]),
  context=tf.constant([""]))
index = simpleneighbors.SimpleNeighbors(
    len(encodings['outputs'][0]), metric='angular')#metric='angular')

print("Embidding dimension: {}".format(len(encodings['outputs'][0])))
#print('Computing embeddings for %s sentences' % len(sentences))
k = 0
for ind, row in data.iterrows():
  #print("row['profile_desc']\n{}".format(row["profile_desc"]))
  
  #main, context = extract_main_and_context(row["motivo_name"])
  encodings = model.signatures['response_encoder'](
    input=tf.constant([row["motivo_name"]]),
    #context=tf.constant([row["context_words"]])
    context=tf.constant([""])
  )
  
  #for i in range(len(response_batch)):
  index.add_one(row["motivo_id"], encodings['outputs'][0])
  motivo_id_to_motivo_name[row["motivo_id"]] = row["motivo_name"]
  k += 1
  print("k: {}".format(k))

start_time = time.time()
index.build()
print("Time to build the index: {}".format(time.time() - start_time))

Embidding dimension: 512
k: 1
k: 2
k: 3
k: 4
k: 5
k: 6
k: 7
k: 8
k: 9
k: 10
k: 11
k: 12
k: 13
k: 14
k: 15
Time to build the index: 0.003645181655883789


**Clasification loop**

In [ ]:
t = time.time()

question = "jilberto"
query_embedding = model.signatures['question_encoder'](tf.constant(question))['outputs'][0]
    
num_results = 3
mot_pred = index.nearest(query_embedding, n=num_results)
#v1 es query_embedding
#v2 es 

print(mot_pred)
print("solucion más cercana")
print( data.iloc[[mot_pred[0]-1]])
#imprir los resultados del df
# do stuff
elapsed = time.time() - t
print('----------')
print('segundos de la consulta')
print(elapsed)

[8, 3, 2]
solucion más cercana
    motivo_name  motivo_id    Pais Genero
7  Gilberto Gil          8  Brasil    MPB
----------
segundos de la consulta
0.16854238510131836


In [ ]:
data['motivo_name'][14]

'Es la persona que pertenece a la fuerza policial y trata de prevenir que se comentan crímenes'

In [ ]:
data

,motivo_name,motivo_id,Empleo
0,Es la persona que trabaja con el dinero y las ...,1,Contador
1,Es la persona que actúa en una obra o película,2,Actor
2,s la persona que diseña casas y edificios,3,Arquitecto
3,Es la persona que estudia las estrellas y el u...,4,Astrónomo
4,Es la persona que produce panes y tortas. Norm...,5,Panadero
5,"Es la persona que trabaja con carne, cortándol...",6,Carnicero
6,"Es la persona que fabrica objetos con madera, ...",7,Carpintero
7,Es la persona que puede arreglar los problemas...,8,Dentista
8,Es la persona que desarrolla soluciones para p...,9,Ingenieros
9,"Es la persona que trabaja en una granja, usual...",10,Granjero
